In [1]:
from astropy.io.fits import getdata, getheader
from nustar_lunar_pointing.orbit import get_epoch_tle, convert_nustar_time, get_moon_j2000

### Load in the attorb data and figure out which TLE row you're going to use.

In [73]:
tlefile = '../data/NUSTAR_TLE_ARCHIVE.txt.2016122'
att = getdata('../data/nu30102028006A.attorb')
hdr = getheader('../data/nu30102028006A.attorb', 1)
mjdref = hdr['MJDREFI']

checktime = convert_nustar_time(att[0]['TIME'])
mindt, line1, line2 = get_epoch_tle(checktime, tlefile)

### Loop over the entries in the attorb file and see if the Moon moves.

Good news, there's less than 3" shift now between using the positions in the attorb file and when you use the TLE to determine the position.

In [74]:
from astropy.coordinates import SkyCoord
doff = []
for ind, t0 in enumerate(att['Time']):
    t1 = convert_nustar_time(t0-5)

    
    ra_moon1, dec_moon1 = get_moon_j2000(t1, line1, line2)
    moon_sky1 = SkyCoord(ra_moon1, dec_moon1)

    position = att[ind]['Position']
    ra_moon2, dec_moon2 = get_moon_j2000(t1, line1, line2, position=position)
    moon_sky2 = SkyCoord(ra_moon2, dec_moon2)
    
    doff.extend([moon_sky1.separation(moon_sky2).arcsec])
    
    if ind > 100:
        break

print(doff)

[3.6305047005545035, 3.949366537089424, 3.6387196883943265, 3.642777594977645, 3.6469103081636924, 3.6510041863046077, 3.6551531662694563, 3.6592437190849183, 3.6633917834649887, 3.9853964789955567, 3.6715778842833924, 3.675603682675079, 3.679686835184939, 3.683748399914219, 3.687870122568211, 3.691985827132534, 3.6960049830738915, 3.7000765758012033, 3.7040690846993383, 3.708172645530046, 3.712177658562114, 3.7162983942037076, 3.7203702418604756, 3.724421253038198, 3.728435673631506, 3.7324861913611262, 3.7364341267644945, 3.740462492531553, 4.070406959895392, 3.748601887371396, 3.752632871964637, 3.7566196539810353, 3.7606013392251443, 3.764555368422063, 3.768568352582263, 3.772604076219319, 4.105786763800521, 3.780600521895563, 3.784631031006597, 3.788601736896683, 3.7924972066652325, 3.796457543571179, 3.800412782834013, 3.804490558632781, 4.140912306746032, 3.812398879071754, 3.8163552835791337, 3.820310115819376, 3.824174374318764, 3.828141015046826, 3.8320671364415717, 3.8360513

### Below was just to figure out that there's a 5 second difference between time timestamps in the attorb file and the times that are actually used to report the satellite position.

In [69]:
from nustar_lunar_pointing.orbit import get_nustar_location
from numpy import sqrt

roff = []
for ind, t0 in enumerate(att['Time']):
    t1 = convert_nustar_time(t0-5)

    tle_position = get_nustar_location(t1, line1, line2)
    position = att[ind]['Position']
        
    dx = tle_position - position
    dr = 0.

    for val in dx:
        dr += val**2.
        
    roff.extend([sqrt(dr)])
    
    
    if ind > 10:
        break